In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import pandas as pd
from pathlib import Path

import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

In [5]:
ROOT_PATH = Path("/content/drive/MyDrive/blog/PyTorch_using_transformers")
DATA_PATH = ROOT_PATH / "datasets"
SAVE_PATH = ROOT_PATH / "03_파이토치 기초/save"
os.makedirs(SAVE_PATH, exist_ok=True)

# 퍼셉트론 모델 실습

x1, x2는 입력값을 의미하며, y는 XOR 게이트를 통과했을 때의 결과를 의미한다.

In [6]:
pd.read_csv(DATA_PATH / 'perceptron.csv')

,x1,x2,y
0,True,True,False
1,True,False,True
2,True,False,True
3,True,True,False
4,True,True,False
...,...,...,...
995,True,True,False
996,True,True,False
997,False,True,True
998,False,False,False


## 단층 퍼셉트론 구조

In [9]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.y = df.iloc[:, 2].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.tensor([self.x1[index], self.x2[index]], dtype=torch.float)
        y = torch.tensor([self.y[index]], dtype=torch.float)
        return x, y

    def __len__(self):
        return self.length

In [10]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer = nn.Sequential(
            nn.Linear(2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer(x)
        return x

In [11]:
train_dataset = CustomDataset(DATA_PATH / "perceptron.csv")
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [13]:
for epoch in range(1000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 100 == 0:
        print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

Epoch :  100, Cost : 0.693
Epoch :  200, Cost : 0.692
Epoch :  300, Cost : 0.692
Epoch :  400, Cost : 0.692
Epoch :  500, Cost : 0.692
Epoch :  600, Cost : 0.692
Epoch :  700, Cost : 0.692
Epoch :  800, Cost : 0.692
Epoch :  900, Cost : 0.693
Epoch : 1000, Cost : 0.692


단층 퍼셉트론 구조로 XOR 문제를 해결하려고 한다면, 비용이 더 이상 감소되지 않는 것을 확인할 수 있다.

In [14]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor([
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1]
    ]).to(device)
    outputs = model(inputs)

    print("---------")
    print(outputs)
    print(outputs <= 0.5)

---------
tensor([[0.4665],
        [0.4981],
        [0.5023],
        [0.5340]], device='cuda:0')
tensor([[ True],
        [ True],
        [False],
        [False]], device='cuda:0')


모델에 값을 입력했을 때도 출력값이 0.5 내외로 출력돼 학습이 정상적으로 진행되지 않은 것을 확인할 수 있다.

## 다층 퍼셉트론 구조

In [18]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(2, 2),
            nn.Sigmoid()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x

In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [22]:
for epoch in range(1000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 100 == 0:
        print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

Epoch :  100, Cost : 0.693
Epoch :  200, Cost : 0.693
Epoch :  300, Cost : 0.693
Epoch :  400, Cost : 0.690
Epoch :  500, Cost : 0.634
Epoch :  600, Cost : 0.187
Epoch :  700, Cost : 0.056
Epoch :  800, Cost : 0.031
Epoch :  900, Cost : 0.021
Epoch : 1000, Cost : 0.016


학습이 진행될수록 비용이 감소하는 것을 확인할 수 있다.

In [23]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor([
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1]
    ]).to(device)
    outputs = model(inputs)

    print("---------")
    print(outputs)
    print(outputs <= 0.5)

---------
tensor([[0.0150],
        [0.9854],
        [0.9799],
        [0.0135]], device='cuda:0')
tensor([[ True],
        [False],
        [False],
        [ True]], device='cuda:0')


모델에 값을 입력했을 때 출력값이 XOR 게이트의 출력값과 동일하다는 것도 확인할 수 있다.